In [122]:
#for importing, formatting and data manipulation
import pandas as pd
import numpy as np

In [316]:
#import your ASV biom table
biom = pd.read_csv('ASVs_bact.csv')

#import your metadata
md = pd.read_csv('Metadata_OG.csv') #take the habit of avoiding spaces in file names

In [317]:
#melt your ASV table to attach dna metadata to ASVs
biom.rename(columns={'#OTU ID': 'feature_id'}, inplace=True)
md.rename(columns={'ID': 'sample_id'}, inplace=True)


biomelted = biom.melt(id_vars=['feature_id'], var_name='sample_id', value_name='feature_frequency')

In [318]:
#sort the concentrations to check they are all positive
md.sort_values('Concentration')

,sample_id,Sample,Cruise,BA,BAA,Category,TOC,Station_ID,WaterColumn,SizeFraction,...,MSC,MSC_DOS,BioPaper_Grid,BioPaper_ID,Fig1,Concentration,Sample.1,SizeFraction3,WaterColumn3,ID3
297,CES22-S8L-BD-D7-Lb,CES22-S8L-BD-D7-Lb,CES22,No,No,BD,No,8L-BD,D7,Lb,...,No,Day 5-6,EASTERN,CE_8L-BD_D7_L,NaN,-1.6,CES22-S8L-BD-D7-Lb,L,D7,CE_8L-BD_D7
198,CES22-S30L-BD-D8-Lb,CES22-S30L-BD-D8-Lb,CES22,No,No,BD,No,30L-BD,D8,Lb,...,No,Day 15,NORTHERN-BD,CE_30L-BD_D8_L,NaN,0.9,CES22-S30L-BD-D8-Lb,L,D8,CE_30L-BD_D8
120,CES22-S15L-MSC-D1R-BASE,CES22-S15L-MSC-D1R-BASE,CES22,No,No,MSC,No,15L,D1R,BASE,...,Yes,Day 13,EASTERN-MSC,CE_15L_D1R_BASE,NaN,1.0,CES22-S15L-MSC-D1R-BASE,NaN,MSC-D1,CE_15L_D1R
299,CES22-S8L-BD-D8-Lb,CES22-S8L-BD-D8-Lb,CES22,No,No,BD,No,8L-BD,D8,Lb,...,No,Day 5-6,EASTERN,CE_8L-BD_D8_L,NaN,1.8,CES22-S8L-BD-D8-Lb,L,D8,CE_8L-BD_D8
196,CES22-S30L-BD-D7-Lb,CES22-S30L-BD-D7-Lb,CES22,No,No,BD,No,30L-BD,D7,Lb,...,No,Day 15,NORTHERN-BD,CE_30L-BD_D7_L,NaN,1.9,CES22-S30L-BD-D7-Lb,L,D7,CE_30L-BD_D7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,CES22-S9L-80mLb,CES22-S9L-80mLb,CES22,No,No,Station,No,9L,D3,Lb,...,No,Day 5-6,EASTERN,CE_9L_D3_L,NaN,163.7,CES22-S9L-80mLb,L,D3,CE_9L_D3
186,CES22-S30L-25mLb,CES22-S30L-25mLb,CES22,No,No,Station,No,30L,D2,Lb,...,No,Day 15,NORTHERN,CE_30L_D2_L,NaN,167.1,CES22-S30L-25mLb,L,D2,CE_30L_D2
231,CES22-S6L-5mLb,CES22-S6L-5mLb,CES22,No,No,Station,No,6L,D1,Lb,...,No,Day 4,EASTERN,CE_6L_D1_L,NaN,169.0,CES22-S6L-5mLb,L,D1,CE_6L_D1
86,CES22-S12-BA1-Tfinal-LTF-Lb,CES22-S12-BA1-Tfinal-LTF-Lb,CES22,Yes,BA1,Bioassay,No,12L-BA1,NaN,Lb,...,No,NaN,EASTERN-BA,CE_12L-BA1_NA_L,NaN,183.6,CES22-S12-BA1-Tfinal-LTF-Lb,L,NaN,CE_BA1_Tfinal-LTF-b


In [319]:
#remove the samples with a negative DNA concentration because we can't calculate that new feature frequency
samples_to_exclude = ['CE_8L-BD_D7']

#exclude those samples
md_SF = md[~md['ID3'].isin(samples_to_exclude)]

In [320]:
#remove rows where samples where not size fractionated, i.e. base/tray/top
md_SF = md_SF[md_SF['SizeFraction2'].isin(['S', 'L'])]

In [321]:
#make a new column of total [DNA] per sample that were size fractionated and need to be pooled
md_SF['[DNAt]'] = md_SF.groupby(['ID3'])['Concentration'].transform('sum')

In [322]:
#separate small and large size fraction
sep_S = md_SF[md_SF.SizeFraction2 == 'S']
sep_L = md_SF[md_SF.SizeFraction2 == 'L']

In [323]:
#calculate DNA proportion per size fraction
md_SF['DNApr'] = md_SF['Concentration']/md_SF['[DNAt]']

In [324]:
#merge with separated on common columns to get corresponding rel. abundances
#md_SF = md_SF[['sample_id', 'DNApr', '[DNAt]']].copy()
sepSLRA = pd.merge(biomelted, md_SF, on=['sample_id'], how='left') #all_md is the metadata file

#remove the ASVs with a null read count
sepSLRA = sepSLRA[sepSLRA.feature_frequency != 0]

In [325]:
#just check which samples didn't match between the metadata and the ASV table
df2 = sepSLRA.DNApr.isnull().groupby([sepSLRA['sample_id']]).sum().astype(int).reset_index(name='counts')
sep_mismatch = df2[df2.counts != 0]
sep_mismatch #it's only the top, base, ..

,sample_id,counts
126,CES22-S15L-MSC-D1R-BASE,229
127,CES22-S15L-MSC-D1R-TRAY,178
128,CES22-S15L-MSC-D2Y-BASE,317
129,CES22-S15L-MSC-D2Y-TRAY,193
130,CES22-S15L-MSC-D3B-BASE,470
131,CES22-S15L-MSC-D3B-TRAY,440
148,CES22-S28LDay1-MSC-D1R-TOP,226
149,CES22-S28LDay1-MSC-D1R-TRAY,122
150,CES22-S28LDay1-MSC-D2Y-TOP,344
151,CES22-S28LDay1-MSC-D2Y-TRAY,229


In [326]:
#calculate corrected per sample ratio, and corrected feature frequency of de-fractionated samples
sepSLRA['Newfeature_frequency'] = sepSLRA['feature_frequency'] * sepSLRA['DNApr']
sepSLRA['Newff'] = sepSLRA.groupby(['feature_id', 'ID3'])['Newfeature_frequency'].transform('sum')

In [327]:
#remove the rows where there was no size fractionation (base, tray, top..)
sepSLRA = sepSLRA[sepSLRA['Newff'].notna()]

In [328]:
#make a new id for the new combined samples
sepSLRA['sampleid'] = sepSLRA['ID3'].astype(str) + "SL"

In [329]:
#uncomment the line above if merging smallandlarge
sepSLRA['SizeFraction'] = 'SL'

In [330]:
#rename the columns
sepSLRA.rename(columns={'feature_frequency':'old_feature_frequency'}, inplace=True)
sepSLRA.rename(columns={'Newff':'feature_frequency'}, inplace=True)
sepSLRA = sepSLRA.drop_duplicates()

In [331]:
#recalculate ratios
sepSLRA['Total'] = sepSLRA['feature_frequency'].groupby(sepSLRA['sampleid']).transform('sum')
sepSLRA['ratio'] = sepSLRA['feature_frequency']/sepSLRA['Total'] #calculate the relative abundance of a feature (0-1 scale per sample)
sepSLRA['nASVs'] = sepSLRA['feature_id'].groupby(sepSLRA['sampleid']).transform('nunique') #calculate the number of ASVs per sample

sepSLRA = sepSLRA.drop_duplicates()

In [332]:
sepSLRA.to_csv('CombinedSL_metadata.csv')